Importo le librerie necessarie per il progetto

In [1]:
import os
import csv
import cv2
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
from PIL import Image
import matplotlib.pyplot as plt
import torchvision.transforms as T
import torchvision.models as models
from torch.utils.data import DataLoader
from torchvision.models import resnet101
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, TensorDataset

Recupero il numero delle classi e verifico se la scheda video è impostata come device

In [2]:
train_path = r'C:\Users\alessio\Desktop\Dimarco_Lomonaco_Salemi\BACKREMOVING\AI\train'

num_classes = len(os.listdir(train_path))
print("n° Classi:", num_classes)
dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())

n° Classi: 8
True


Questa funzione prende in input un percorso relativo ad un'immagine (image_path), una bounding box (bbox) che specifica la regione di interesse dell'immagine, e un valore booleano (noback). Se noback è impostato su 0, la funzione esegue la segmentazione dell'immagine per rimuovere lo sfondo intorno alla bounding box utilizzando l'algoritmo GrabCut. Successivamente, l'immagine risultante viene ritagliata in base ai valori della bounding box e normalizzata. 

In [3]:
import cv2
import numpy as np

def crop_image(image_path, bbox, noback):
    # Carica l'immagine dal percorso specificato
    image = cv2.imread(image_path)

    # Estrai i valori x_min, y_min, x_max e y_max dal bounding box
    x_min, y_min, x_max, y_max = bbox

    # Converte l'immagine da BGR a RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Se noback è False (0), esegui la rimozione dello sfondo
    if noback == 0:
        # Crea una maschera inizialmente vuota
        mask = np.zeros(image_rgb.shape[:2], np.uint8)

        # Crea modelli per il background (bgd_model) e il foreground (fgd_model)
        bgd_model = np.zeros((1, 65), np.float64)
        fgd_model = np.zeros((1, 65), np.float64)

        # Definisce un rettangolo che contiene l'oggetto da ritagliare
        rectangle = (x_min, y_min, x_max - x_min, y_max - y_min)

        # Esegue l'algoritmo di GrabCut per separare l'oggetto dallo sfondo
        cv2.grabCut(image_rgb, mask, rectangle, bgd_model, fgd_model, 1, cv2.GC_INIT_WITH_RECT)

        # Crea una maschera per il foreground, dove il foreground è etichettato come 3 o 1
        mask_fg = np.where((mask == 3) | (mask == 1), 255, 0).astype('uint8')

        # Applica la maschera al'immagine per rimuovere lo sfondo
        image_rgb = cv2.bitwise_and(image_rgb, image_rgb, mask=mask_fg)

    # Esegue il ritaglio dell'immagine basato sul bounding box
    cropped_image = image_rgb[y_min:y_max, x_min:x_max]

    # Normalizza i valori dell'immagine dividendoli per 255.0
    res = np.array(cropped_image) / 255.0

    # Restituisce l'immagine ritagliata e, se necessario, senza sfondo
    return res

# Chiamata alla funzione
# risultato = crop_image(image_path, bbox, noback)

Questo codice legge un file CSV contenente informazioni sulle immagini come:path, bounding box e etichette. Successivamente, manda ciascuna immagine in elaborazione alla funzione precedete, dopo memorizza l'immagine elaborata e l'etichetta associata in liste separate per l'addestramento di un modello di machine learning.
Il parametro noback collegato all'etichetta (label) è utilizzato per rimuovere il background solo dalle immagini con la prima etichetta (label 0). Questa strategia è adottata per evitare possibili errori di classificazione delle immagini con questa etichetta durante l'addestramento del modello, poichè le immagini di test hanno il suo stesso sfondo.

In [45]:
# Inizializza due liste vuote per archiviare le immagini di addestramento e le relative etichette.
train_images = []
train_labels = []

# Leggi i dati di addestramento da un file CSV chiamato 'train.csv' utilizzando pandas e memorizzali in un DataFrame.
train_df = pd.read_csv('train.csv')

# Itera su ogni riga del DataFrame 'train_df' per processare i dati di addestramento.
for i, (imp, x_min, y_min, x_max, y_max, label) in train_df.iterrows():
    # Costruisci il percorso completo dell'immagine utilizzando 'imp' e la directory specificata.
    image_path = os.path.join('AI/train', imp.replace('/', '\\'))
    
    # Crea un bounding box (bbox) utilizzando i valori di x_min, y_min, x_max e y_max.
    bbox = x_min, y_min, x_max, y_max
    
    # Chiama la funzione 'crop_image' per ritagliare l'immagine e, se necessario, rimuovere lo sfondo.
    image = crop_image(image_path, bbox, label)
    
    # Verifica che l'immagine ritagliata abbia una dimensione non nulla.
    a, b, c = image.shape
    
    # Se l'immagine ha una dimensione valida (diversa da zero), aggiungi l'etichetta e l'immagine alle liste.
    if a != 0 and b != 0 and c != 0:
        train_labels.append(label)
        train_images.append(image)

Questo codice fa la stessa cosa del precedente, solo che carica le immagini senza sfondo per tutte quelle immagini che hanno label diversa da quella iniziale.
La ragione principale di questa elaborazione è compensare la perdita di qualità che alcune immagini hanno subito durante il caricamento precedente. Questo processo aiuta anche ad ampliare il dataset di addestramento. Inoltre, il fatto di rimuovere la prima classe (label 0) dall'algoritmo di elaborazione delle immagini aiuta a evitare che il modello apprenda erroneamente il background, riducendo così il rischio di errori di classificazione causati dallo sfondo delle immagini.

In [46]:
# Leggi i dati di addestramento da un file CSV chiamato 'train.csv' utilizzando pandas e memorizzali in un DataFrame.
train_df = pd.read_csv('train.csv')

# Itera su ogni riga del DataFrame 'train_df' per processare i dati di addestramento.
for i, (imp, x_min, y_min, x_max, y_max, label) in train_df.iterrows():
    # Verifica se il valore dell'etichetta 'label' è diverso da 0.
    if label != 0:
        # Costruisci il percorso completo dell'immagine utilizzando 'imp' e la directory specificata.
        image_path = os.path.join('AI/train', imp.replace('/', '\\'))
        
        # Crea un bounding box (bbox) utilizzando i valori di x_min, y_min, x_max e y_max.
        bbox = x_min, y_min, x_max, y_max
        
        # Chiama la funzione 'crop_image' per ritagliare l'immagine e rimuovere lo sfondo (con noback=0).
        image = crop_image(image_path, bbox, 0)
        
        # Verifica che l'immagine ritagliata abbia una dimensione non nulla.
        a, b, c = image.shape
        
        # Se l'immagine ha una dimensione valida (diversa da zero), aggiungi l'etichetta e l'immagine alle liste.
        if a != 0 and b != 0 and c != 0:
            train_labels.append(label)
            train_images.append(image)

Questo codice converte le immagini e le etichette del dataset in tensori PyTorch, necessari per l'addestramento di modelli di deep learning. Inoltre, riorganizza le dimensioni delle immagini per adattarle al formato comunemente utilizzato da PyTorch.

In [47]:
# Converti le liste 'train_images' e 'train_labels' in array NumPy.
train_images = np.array(train_images)
train_labels = np.array(train_labels)

# Trova l'insieme unico di etichette presenti nei dati di addestramento.
labels = list(set(train_labels))

# Stampa le etichette uniche presenti nei dati.
print("labels:", labels)

# Conversione delle immagini in tensori PyTorch:
# - `train_images` viene convertito in un tensore e la disposizione dei canali dell'immagine viene modificata da NHWC (numero campioni x altezza x larghezza x canali) a NCHW.
# - `train_labels` viene convertito in un tensore di tipo `long`.
train_images_tensor = torch.from_numpy(train_images).permute(0, 3, 1, 2).float()
train_labels_tensor = torch.from_numpy(train_labels).long()

labels: [0, 1, 2, 3, 4, 5, 6, 7]


Imposto la batch size e creo il dataset di addestramento, di validazione e i data loader.

In [107]:
# Definizione della dimensione del batch
batch_size = 32

# Divisione dei dati in set di addestramento e set di validazione
X_train, X_val, Y_train, Y_val = train_test_split(
    train_images_tensor, train_labels_tensor, test_size=0.2, random_state=42
)

# Creazione dei dataset utilizzando TensorDataset da PyTorch
train_dataset = TensorDataset(X_train, Y_train)
val_dataset = TensorDataset(X_val, Y_val)

# Creazione dei data loader per caricare i dati in batch durante l'addestramento
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)


Creo un modello di classificazione delle immagini basato su ResNet-101. I pesi del modello sono scongelati per consentire l'apprendimento dai dati di addestramento. Il classificatore è sostituito con uno personalizzato per adattarlo al nostro problema di classificazione.
La seconda parte è dedicata al training del modello dove viene monitorata l'accuratezza e la perdita sia sui dati di addestramento che di convalida.

In [4]:
# Definizione delle trasformazioni per il data augmentation
transform = T.Compose([
    T.Resize(224, antialias=None),  # Ridimensiona le immagini a 224x224 pixel
    T.RandomHorizontalFlip()  # Applica una trasformazione di flip orizzontale casuale alle immagini
])

# Caricamento di un modello preallenato ResNet con pesi addestrati su ImageNet
model = resnet101(weights="IMAGENET1K_V2")

# Modifica il classificatore finale del modello per adattarlo al problema specifico
n_inputs = model.fc.in_features  # Ottiene il numero di feature in input al classificatore
model.fc = nn.Sequential(
    nn.Linear(n_inputs, 2048),  # Aggiunge un layer completamente connesso (fully connected)
    nn.BatchNorm1d(2048),  # Aggiunge normalizzazione BatchNorm
    nn.ReLU(),  # Aggiunge una funzione di attivazione ReLU
    nn.Dropout(0.5),  # Aggiunge dropout con probabilità 0.5 per la regolarizzazione
    nn.Linear(2048, 2048),  # Secondo layer completamente connesso
    nn.BatchNorm1d(2048),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(2048, num_classes),  # Output layer con il numero di classi desiderato
    nn.LogSoftmax(dim=1)  # Applica il logaritmo softmax per ottenere le probabilità delle classi
)

# Scongela tutti i pesi del modello in modo che possa imparare dai dati di addestramento
for param in model.parameters():
    param.requires_grad = True

# Definizione della funzione di costo per l'addestramento, in questo caso NLLLoss (Negative Log Likelihood Loss)
criterion = nn.NLLLoss()

In [50]:
# Sposta il modello sulla GPU specificata (dev) per l'addestramento
model = model.to(dev)

# Definizione della funzione di loss e dell'ottimizzatore
lr = 0.0001  # Tasso di apprendimento
optimizer = torch.optim.Adam(model.parameters(), lr=lr)  # Ottimizzatore Adam
# Definizione del learning rate scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.01)
best_val_accuracy = 0.0
best_val_loss = 0.0
epoch = 0

# Loop di addestramento (può essere interrotto manualmente)
try:
    while True:
        # Fase di addestramento
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        for batch_images, batch_labels in train_loader:
            batch_images = transform(batch_images)  # Applica le trasformazioni alle immagini
            batch_images = batch_images.to("cuda")  # Sposta le immagini sulla GPU
            batch_labels = batch_labels.to("cuda")  # Sposta le etichette sulla GPU
            optimizer.zero_grad()  # Azzera i gradienti
            outputs = model(batch_images)  # Passa le immagini attraverso il modello
            loss = criterion(outputs, batch_labels)  # Calcola la loss
            loss.backward()  # Calcola i gradienti
            optimizer.step()  # Esegue l'ottimizzazione, aggiorna i pesi
            _, predicted = torch.max(outputs.data, 1)
            train_total += batch_labels.size(0)
            train_correct += (predicted == batch_labels).sum().item()
            train_loss += loss.item() * batch_images.size(0)

        train_accuracy = train_correct / train_total
        train_loss /= len(train_loader.dataset)

        # Fase di convalida
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for batch_images, batch_labels in val_loader:
                batch_images = transform(batch_images)  # Applica le trasformazioni alle immagini
                batch_images = batch_images.to("cuda")  # Sposta le immagini sulla GPU
                batch_labels = batch_labels.to("cuda")  # Sposta le etichette sulla GPU
                outputs = model(batch_images)  # Passa le immagini attraverso il modello
                _, predicted = torch.max(outputs.data, 1)
                val_loss += criterion(outputs, batch_labels)
                val_total += batch_labels.size(0)
                val_correct += (predicted == batch_labels).sum().item()

        val_accuracy = val_correct / val_total
        val_loss /= len(val_loader)
        scheduler.step(val_accuracy)  # Aggiorna il tasso di apprendimento dinamicamente
        torch.save(model.state_dict(), "model_1_" + str(epoch) + ".pth")  # Salva il modello
        print(f"Epoch [{epoch+1}] Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.2%}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2%}")

        epoch += 1
        torch.cuda.empty_cache()  # Svuota la cache della GPU per evitare memory leak

except KeyboardInterrupt:
    print("Addestramento interrotto manualmente")
    torch.save(model.state_dict(), "model_1_lastSave.pth")  # Salva l'ultimo modello in caso di interruzione manuale

Epoch [1] Training Loss: 0.4803, Training Accuracy: 86.10%, Validation Loss: 0.0384, Validation Accuracy: 99.00%
Epoch [2] Training Loss: 0.0339, Training Accuracy: 99.50%, Validation Loss: 0.0273, Validation Accuracy: 99.33%
Epoch [3] Training Loss: 0.0224, Training Accuracy: 99.46%, Validation Loss: 0.0345, Validation Accuracy: 99.17%
Epoch [4] Training Loss: 0.0138, Training Accuracy: 99.71%, Validation Loss: 0.0169, Validation Accuracy: 99.33%
Epoch [5] Training Loss: 0.0129, Training Accuracy: 99.75%, Validation Loss: 0.0262, Validation Accuracy: 99.17%
Epoch [6] Training Loss: 0.0145, Training Accuracy: 99.67%, Validation Loss: 0.0242, Validation Accuracy: 99.33%
Epoch [7] Training Loss: 0.0197, Training Accuracy: 99.37%, Validation Loss: 0.0175, Validation Accuracy: 99.17%
Epoch [8] Training Loss: 0.0139, Training Accuracy: 99.71%, Validation Loss: 0.0161, Validation Accuracy: 99.33%
Epoch [9] Training Loss: 0.0117, Training Accuracy: 99.75%, Validation Loss: 0.0166, Validation 

Funzione che legge un path da un file csv, prende l'immagine associata e ne predice il risultato

In [7]:
def model_ev(model_path):
    # Definizione delle trasformazioni per il data pre-processing
    data_transforms = T.Compose([
        T.ToTensor(),  # Converte l'immagine in un tensore PyTorch
        T.Resize(224, antialias=None),  # Ridimensiona l'immagine a 224x224 pixel
    ])
    
    label_pre = []  # Lista per memorizzare le previsioni delle etichette
    model.load_state_dict(torch.load(model_path))  # Carica i pesi del modello precedentemente addestrato
    model.to("cuda")  # Sposta il modello sulla GPU
    model.eval()  # Imposta il modello in modalità di valutazione (senza addestramento)
    test_df = pd.read_csv('test.csv')  # Legge il file CSV di dati di test
    output_file = 'submission.csv'  # Nome del file di output
    with open(output_file, mode='w', newline='') as file:
        writer = csv.writer(file)  # Creazione di un writer per il file CSV di output
        writer.writerow(['image', 'class'])  # Scrive l'intestazione nel file di output

        for i, (image_path, x_min, y_min, x_max, y_max) in test_df.iterrows():
            bbox = x_min, y_min, x_max, y_max
            image_ = os.path.join('AI/test', image_path.replace('/', '\\'))  # Costruzione del percorso dell'immagine di test
            image = crop_image(image_, bbox, 1)  # Ritaglia l'immagine
            image = data_transforms(image)  # Applica le trasformazioni all'immagine
            image = image.float()  # Converte il tensore in tipo float
            image = image.unsqueeze(0)  # Aggiunge una dimensione per rappresentare il batch (batch size 1)
            image = image.cuda()  # Sposta l'immagine sulla GPU

            with torch.no_grad():
                outputs = model(image)  # Esegue il modello per ottenere le previsioni
                _, predicted = torch.max(outputs.data, 1)  # Trova la classe predetta
                predicted_label = predicted.item()  # Estrae l'etichetta predetta come valore intero
                label_pre.append(predicted_label)  # Aggiunge l'etichetta predetta alla lista
                writer.writerow([image_path, predicted_label])  # Scrive l'immagine e l'etichetta nel file di output

    return label_pre  # Restituisce la lista delle etichette previste

Funzione che ritorna la percentuale di accuratezza sul test

In [5]:
def test(label_pre):
    # Carica il file CSV delle classi
    class_df = pd.read_csv('class.csv')
    
    i = 0  # Inizializza un contatore per tenere traccia dell'indice delle etichette previste
    correct = 0  # Inizializza un contatore per tenere traccia del numero di previsioni corrette
    
    # Itera attraverso le righe del file CSV delle classi
    for label in class_df.iterrows():
        # Confronta l'etichetta reale (da class_df) con l'etichetta prevista (da label_pre)
        if int(label[1].values) == label_pre[i]:
            correct = correct + 1  # Incrementa il contatore se la previsione è corretta
        else:
            print(i)  # Stampa l'indice dell'etichetta in caso di previsione errata
           
        i = i + 1  # Passa all'indice successivo
    
    # Calcola l'accuratezza (il rapporto tra le previsioni corrette e il totale delle previsioni)
    accuracy = correct / i
    
    return accuracy  # Restituisce l'accuratezza calcolata

Prendiamo il modello che ha le migliori performance nella fase precedente, ad occhio si puo vedere che il numero 2, dopo di questa infatti ci sono leggeri segni di overfitting.
Sotto si allena il modello scelto con un lr piu basso e aggiunto un fattore di regolarizzazione. Il modello scelto ha ottenuto 99.25% sul test.

In [88]:
model_path = 'model_1_1.pth'
model.load_state_dict(torch.load(model_path))  # Carica i pesi del modello preaddestrato

model = model.to(dev)  # Sposta il modello sulla GPU specificata (dev)

# Definizione della funzione di loss e dell'ottimizzatore con un nuovo tasso di apprendimento (lr) e regolarizzazione (weight_decay)
lr = 1e-7  # Tasso di apprendimento
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=0.01)  # Ottimizzatore Adam con regolarizzazione L2 (weight decay)
best_val_accuracy = 0.0
best_val_loss = 0.0
epoch = 0

# Loop di addestramento (può essere interrotto manualmente)
try:
    while True:
        # Fase di addestramento
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        for batch_images, batch_labels in train_loader:
            batch_images = transform(batch_images)  # Applica le trasformazioni alle immagini
            batch_images = batch_images.to("cuda")  # Sposta le immagini sulla GPU
            batch_labels = batch_labels.to("cuda")  # Sposta le etichette sulla GPU
            optimizer.zero_grad()  # Azzera i gradienti
            outputs = model(batch_images)  # Passa le immagini attraverso il modello
            loss = criterion(outputs, batch_labels)  # Calcola la loss
            loss.backward()  # Calcola i gradienti
            optimizer.step()  # Esegue l'ottimizzazione
            _, predicted = torch.max(outputs.data, 1)
            train_total += batch_labels.size(0)
            train_correct += (predicted == batch_labels).sum().item()
            train_loss += loss.item() * batch_images.size(0)

        train_accuracy = train_correct / train_total
        train_loss /= len(train_loader.dataset)

        # Fase di convalida
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for batch_images, batch_labels in val_loader:
                batch_images = transform(batch_images)  # Applica le trasformazioni alle immagini
                batch_images = batch_images.to("cuda")  # Sposta le immagini sulla GPU
                batch_labels = batch_labels.to("cuda")  # Sposta le etichette sulla GPU
                outputs = model(batch_images)  # Passa le immagini attraverso il modello
                _, predicted = torch.max(outputs.data, 1)
                val_loss += criterion(outputs, batch_labels)
                val_total += batch_labels.size(0)
                val_correct += (predicted == batch_labels).sum().item()

        val_accuracy = val_correct / val_total
        val_loss /= len(val_loader)
        scheduler.step(val_accuracy)  # Aggiorna il tasso di apprendimento dinamicamente
        torch.save(model.state_dict(), "model_1_1_" + str(epoch) + ".pth")  # Salva il modello
        print(f"Epoch [{epoch+1}] Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.2%}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2%}")

        epoch += 1
        torch.cuda.empty_cache()  # Svuota la cache della GPU per evitare memory leak

except KeyboardInterrupt:
    print("Addestramento interrotto manualmente")
    torch.save(model.state_dict(), "model_1.1_lastSave.pth")  # Salva l'ultimo modello in caso di interruzione manuale

Epoch [1] Training Loss: 0.0238, Training Accuracy: 99.50%, Validation Loss: 0.0172, Validation Accuracy: 99.58%
Epoch [2] Training Loss: 0.0186, Training Accuracy: 99.72%, Validation Loss: 0.0178, Validation Accuracy: 99.58%
Epoch [3] Training Loss: 0.0249, Training Accuracy: 99.39%, Validation Loss: 0.0170, Validation Accuracy: 99.58%
Epoch [4] Training Loss: 0.0211, Training Accuracy: 99.55%, Validation Loss: 0.0171, Validation Accuracy: 99.58%
Epoch [5] Training Loss: 0.0184, Training Accuracy: 99.72%, Validation Loss: 0.0179, Validation Accuracy: 99.58%
Epoch [6] Training Loss: 0.0180, Training Accuracy: 99.67%, Validation Loss: 0.0184, Validation Accuracy: 99.58%
Epoch [7] Training Loss: 0.0200, Training Accuracy: 99.78%, Validation Loss: 0.0162, Validation Accuracy: 99.58%
Epoch [8] Training Loss: 0.0197, Training Accuracy: 99.55%, Validation Loss: 0.0169, Validation Accuracy: 99.58%
Epoch [9] Training Loss: 0.0247, Training Accuracy: 99.39%, Validation Loss: 0.0172, Validation 

Poichè le prestazione sembrano molto simili e non c'è evidenza chiara di overfitting, consideriamo l'epoca 7 come la migliore tra quelle fornite, poiché ha la validation loss più bassa. 99.625 sul test;
Diminuiamo ulteriolmente il lr.

In [98]:
model_path = 'model_1_1_6.pth'
model.load_state_dict(torch.load(model_path))  # Carica i pesi del modello preaddestrato

model = model.to(dev)  # Sposta il modello sulla GPU specificata (dev)

# Definizione della funzione di loss e dell'ottimizzatore con un nuovo tasso di apprendimento (lr) e regolarizzazione (weight_decay)
lr = 1e-10  # Tasso di apprendimento molto basso
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=0.1)  # Ottimizzatore Adam con regolarizzazione L2 (weight decay)
best_val_accuracy = 0.0
best_val_loss = 0.0
epoch = 0

# Loop di addestramento (può essere interrotto manualmente)
try:
    while True:
        # Fase di addestramento
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        for batch_images, batch_labels in train_loader:
            batch_images = transform(batch_images)  # Applica le trasformazioni alle immagini
            batch_images = batch_images.to("cuda")  # Sposta le immagini sulla GPU
            batch_labels = batch_labels.to("cuda")  # Sposta le etichette sulla GPU
            optimizer.zero_grad()  # Azzera i gradienti
            outputs = model(batch_images)  # Passa le immagini attraverso il modello
            loss = criterion(outputs, batch_labels)  # Calcola la loss
            loss.backward()  # Calcola i gradienti
            optimizer.step()  # Esegue l'ottimizzazione
            _, predicted = torch.max(outputs.data, 1)
            train_total += batch_labels.size(0)
            train_correct += (predicted == batch_labels).sum().item()
            train_loss += loss.item() * batch_images.size(0)

        train_accuracy = train_correct / train_total
        train_loss /= len(train_loader.dataset)

        # Fase di convalida
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for batch_images, batch_labels in val_loader:
                batch_images = transform(batch_images)  # Applica le trasformazioni alle immagini
                batch_images = batch_images.to("cuda")  # Sposta le immagini sulla GPU
                batch_labels = batch_labels.to("cuda")  # Sposta le etichette sulla GPU
                outputs = model(batch_images)  # Passa le immagini attraverso il modello
                _, predicted = torch.max(outputs.data, 1)
                val_loss += criterion(outputs, batch_labels)
                val_total += batch_labels.size(0)
                val_correct += (predicted == batch_labels).sum().item()

        val_accuracy = val_correct / val_total
        val_loss /= len(val_loader)
        scheduler.step(val_accuracy)  # Aggiorna il tasso di apprendimento dinamicamente
        torch.save(model.state_dict(), "model_1.3." + str(epoch) + ".pth")  # Salva il modello
        print(f"Epoch [{epoch+1}] Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.2%}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2%}")

        epoch += 1
        torch.cuda.empty_cache()  # Svuota la cache della GPU per evitare memory leak

except KeyboardInterrupt:
    print("Addestramento interrotto manualmente")
    torch.save(model.state_dict(), "model_1.3_lastSave.pth")  # Salva l'ultimo modello in caso di interruzione manuale

Epoch [1] Training Loss: 0.0188, Training Accuracy: 99.72%, Validation Loss: 0.0181, Validation Accuracy: 99.58%
Epoch [2] Training Loss: 0.0173, Training Accuracy: 99.67%, Validation Loss: 0.0172, Validation Accuracy: 99.58%
Epoch [3] Training Loss: 0.0205, Training Accuracy: 99.50%, Validation Loss: 0.0170, Validation Accuracy: 99.58%
Epoch [4] Training Loss: 0.0215, Training Accuracy: 99.55%, Validation Loss: 0.0176, Validation Accuracy: 99.58%
Epoch [5] Training Loss: 0.0202, Training Accuracy: 99.72%, Validation Loss: 0.0170, Validation Accuracy: 99.58%
Epoch [6] Training Loss: 0.0196, Training Accuracy: 99.61%, Validation Loss: 0.0172, Validation Accuracy: 99.58%
Epoch [7] Training Loss: 0.0201, Training Accuracy: 99.67%, Validation Loss: 0.0174, Validation Accuracy: 99.58%
Epoch [8] Training Loss: 0.0172, Training Accuracy: 99.78%, Validation Loss: 0.0173, Validation Accuracy: 99.58%
Epoch [9] Training Loss: 0.0183, Training Accuracy: 99.72%, Validation Loss: 0.0176, Validation 

Qui la scelta ricadeva tra la 2 e la 5, abbiamo scelto la 2 poichè la differenza tra le loss è minima. 99.75 sul test;
Poichè i risultati sono gia buoni teniamo questi pesi e cerchiamo di allenare meglio solo il classificatore per vedere se aumentano le performance.

In [167]:
model_path = 'model_1.3.1.pth'
model.load_state_dict(torch.load(model_path))  # Carica i pesi del modello preaddestrato

# Imposta tutti i parametri del modello come non addestrabili (congelati)
for param in model.parameters():
    param.requires_grad = False

# Modifica il classificatore finale del modello con un nuovo strato
model.fc = nn.Sequential(
    nn.Linear(n_inputs, 2048),
    nn.BatchNorm1d(2048),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(2048, 2048),
    nn.BatchNorm1d(2048),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(2048, num_classes),
    nn.LogSoftmax(dim=1)
)

model = model.to(dev)  # Sposta il modello sulla GPU specificata (dev)

# Definizione della funzione di loss e dell'ottimizzatore con un nuovo tasso di apprendimento (lr) e regolarizzazione (weight_decay)
lr = 1e-5  # Tasso di apprendimento
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=0.1)  # Ottimizzatore Adam con regolarizzazione L2 (weight decay)
best_val_accuracy = 0.0
best_val_loss = 0.0
epoch = 0

# Loop di addestramento (può essere interrotto manualmente)
try:
    while True:
        # Fase di addestramento
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        for batch_images, batch_labels in train_loader:
            batch_images = transform(batch_images)  # Applica le trasformazioni alle immagini
            batch_images = batch_images.to("cuda")  # Sposta le immagini sulla GPU
            batch_labels = batch_labels.to("cuda")  # Sposta le etichette sulla GPU
            optimizer.zero_grad()  # Azzera i gradienti
            outputs = model(batch_images)  # Passa le immagini attraverso il modello
            loss = criterion(outputs, batch_labels)  # Calcola la loss
            loss.backward()  # Calcola i gradienti
            optimizer.step()  # Esegue l'ottimizzazione
            _, predicted = torch.max(outputs.data, 1)
            train_total += batch_labels.size(0)
            train_correct += (predicted == batch_labels).sum().item()
            train_loss += loss.item() * batch_images.size(0)

        train_accuracy = train_correct / train_total
        train_loss /= len(train_loader.dataset)

        # Fase di convalida
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for batch_images, batch_labels in val_loader:
                batch_images = transform(batch_images)  # Applica le trasformazioni alle immagini
                batch_images = batch_images.to("cuda")  # Sposta le immagini sulla GPU
                batch_labels = batch_labels.to("cuda")  # Sposta le etichette sulla GPU
                outputs = model(batch_images)  # Passa le immagini attraverso il modello
                _, predicted = torch.max(outputs.data, 1)
                val_loss += criterion(outputs, batch_labels)
                val_total += batch_labels.size(0)
                val_correct += (predicted == batch_labels).sum().item()

        val_accuracy = val_correct / val_total
        val_loss /= len(val_loader)
        scheduler.step(val_accuracy)  # Aggiorna il tasso di apprendimento dinamicamente
        torch.save(model.state_dict(), "model_1.4." + str(epoch) + ".pth")  # Salva il modello
        print(f"Epoch [{epoch+1}] Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.2%}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2%}")

        epoch += 1
        torch.cuda.empty_cache()  # Svuota la cache della GPU per evitare memory leak

except KeyboardInterrupt:
    print("Training interrotto manualmente")
    torch.save(model.state_dict(), "model_1.4_lastSave.pth")  # Salva l'ultimo modello in caso di interruzione manuale

Epoch [1] Training Loss: 1.2966, Training Accuracy: 67.23%, Validation Loss: 0.5596, Validation Accuracy: 99.05%
Epoch [2] Training Loss: 0.4530, Training Accuracy: 97.79%, Validation Loss: 0.2154, Validation Accuracy: 99.24%
Epoch [3] Training Loss: 0.2376, Training Accuracy: 99.02%, Validation Loss: 0.1364, Validation Accuracy: 99.33%
Epoch [4] Training Loss: 0.1829, Training Accuracy: 99.38%, Validation Loss: 0.1017, Validation Accuracy: 99.24%
Epoch [5] Training Loss: 0.1362, Training Accuracy: 99.28%, Validation Loss: 0.0845, Validation Accuracy: 99.43%
Epoch [6] Training Loss: 0.1056, Training Accuracy: 99.64%, Validation Loss: 0.0753, Validation Accuracy: 99.33%
Epoch [7] Training Loss: 0.1027, Training Accuracy: 99.54%, Validation Loss: 0.0641, Validation Accuracy: 99.24%
Epoch [8] Training Loss: 0.0900, Training Accuracy: 99.44%, Validation Loss: 0.0583, Validation Accuracy: 99.33%
Epoch [9] Training Loss: 0.0819, Training Accuracy: 99.28%, Validation Loss: 0.0495, Validation 

Senza ombra di dubbio sembra che la miglio epoca sia la 31 poiche dopo di questa ci sono leggeri segnali di overfitting, testiamola:
e creiamo il file submission per il test finale su kaagle

In [8]:
model_path = 'model_1.4.30.pth'  # Percorso del modello addestrato da utilizzare per le previsioni
label_pre = model_ev(model_path)  # Esegue le previsioni utilizzando il modello

# Calcola l'accuratezza dei risultati
test_result = test(label_pre)

496


0.99875